In [53]:
import torch
from torch import nn

In [54]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [55]:
device

'cuda'

In [56]:
from zipfile import ZipFile
import requests
from pathlib import Path
import os

data_path = Path("data")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir() :
    print("Folder already exist,no need to download")
else:
    data_path.mkdir(parents=True,exist_ok=True)
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        print("Downloading data...")
        res = requests.get(
            "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip"
        )
        f.write(res.content)

    with ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zf:
        print("Extract file...")
        zf.extractall(image_path)

    os.remove(data_path / "pizza_steak_sushi.zip")

Folder already exist,no need to download


In [57]:
train_dir = image_path / "train"
test_dir = image_path / "test"

In [58]:
import os

for dirpath,dirnames,filenames in os.walk(train_dir):
    print(f"on {dirpath} has {len(dirnames)} directories and {len(filenames)} files")

on data\pizza_steak_sushi\train has 3 directories and 0 files
on data\pizza_steak_sushi\train\pizza has 0 directories and 154 files
on data\pizza_steak_sushi\train\steak has 0 directories and 146 files
on data\pizza_steak_sushi\train\sushi has 0 directories and 150 files


In [59]:
## Manual Transforms
from torchvision.transforms import v2

from torchvision.transforms.functional import InterpolationMode


manual_transforms = v2.Compose(

    [

        v2.ToImage(),

        v2.Resize(size=(256, 256), interpolation=InterpolationMode.BICUBIC),

        v2.CenterCrop(size=(224, 224)),

        v2.ToDtype(dtype=torch.float32, scale=True),

        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

    ]
)

In [60]:
## Auto Transforms
from torchvision.models.efficientnet import EfficientNet_B0_Weights

weights  = EfficientNet_B0_Weights.DEFAULT

weights

EfficientNet_B0_Weights.IMAGENET1K_V1

In [61]:
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [62]:
BATCH_SIZE = 32

In [63]:
from going_modular.data_setup import create_dataloaders

train_dataloader, test_dataloader, class_names = create_dataloaders(
    train_dir, test_dir, manual_transforms, auto_transforms, BATCH_SIZE
)

len(train_dataloader)

15

In [66]:
from torchvision.models.efficientnet import efficientnet_b0,EfficientNet_B0_Weights

weights = EfficientNet_B0_Weights.DEFAULT
model = efficientnet_b0(weights=weights)

In [67]:
from torchinfo import summary

summary(model=model,input_size=(BATCH_SIZE,3,256,256))

Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [32, 1000]                --
├─Sequential: 1-1                                       [32, 1280, 8, 8]          --
│    └─Conv2dNormActivation: 2-1                        [32, 32, 128, 128]        --
│    │    └─Conv2d: 3-1                                 [32, 32, 128, 128]        864
│    │    └─BatchNorm2d: 3-2                            [32, 32, 128, 128]        64
│    │    └─SiLU: 3-3                                   [32, 32, 128, 128]        --
│    └─Sequential: 2-2                                  [32, 16, 128, 128]        --
│    │    └─MBConv: 3-4                                 [32, 16, 128, 128]        1,448
│    └─Sequential: 2-3                                  [32, 24, 64, 64]          --
│    │    └─MBConv: 3-5                                 [32, 24, 64, 64]          6,004
│    │    └─MBConv: 3-6                              

In [73]:
from torch import nn
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2,inplace=True),
    nn.Linear(in_features=1280,out_features=len(class_names),bias=True)
)

In [74]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=3, bias=True)
)